# Statistics, Randomness, and Probability


## Introduction

In this chapter, you'll learn about doing statistics with code.

This chapter uses the TODO, along with some others that we've already seen. If you're running this code, you may need to install these packages using, for example, `pip install packagename` on your computer's command line. (If you're not sure what a command line or terminal is, take a quick look at the basics of coding chapter.)

### Notation

Greek letters, like $\beta$, are the truth. Modified Greek letters are an estimate of the truth, for example $\hat{\beta}$. Letters from the Latin alphabet denote the values of data, for instance $x$ for a variable or vector. Modified Latin alphabet letters denote computations performed on data, for instance $\bar{x} = \frac{1}{n} \displaystyle\sum_{i} x_i$ where $n$ is number of samples.

### Imports

First we need to import the packages we'll be using

In [ ]:
import numpy as np
from scipy import stats
import pingouin as pg
import matplotlib.pyplot as plt
import pandas as pd

## Basics

Let's start with computing the simplest statistics you can think of using some synthetic data. Many of the functions have lots of extra options that we won't explore here (like weights or normalisation); remember that you can see these using the `help()` method. 

We'll generate a vector with 100 entries:

In [ ]:
data = np.array(range(100))
data

In [ ]:
from myst_nb import glue
import sympy

dict_fns = {'mean': np.mean(data),
            'std': np.std(data),
            'mode': stats.mode([0, 1, 2, 3, 3, 3, 5])[0][0],
            'median': np.median(data)}

for name, eval_fn in dict_fns.items():
    glue(name, f'{eval_fn:.1f}')


# Set max rows displayed for readability
pd.set_option('display.max_rows', 6)
# Plot settings
plot_style = {'xtick.labelsize': 20,
                  'ytick.labelsize': 20,
                  'font.size': 22,
                  'figure.autolayout': True,
                  'figure.figsize': (10, 5.5),
                  'axes.titlesize': 22,
                  'axes.labelsize': 20,
                  'lines.linewidth': 4,
                  'lines.markersize': 6,
                  'legend.fontsize': 16,
                  'mathtext.fontset': 'stix',
                  'font.family': 'STIXGeneral',
                  'legend.frameon': False}
plt.style.use(plot_style)

Okay, let's see how some basic statistics are computed. The mean is `np.mean(data)=`{glue:}`mean`, the standard deviation is `np.std(data)=`{glue:}`std`, and the median is given by `np.median(data)=`{glue:}`median`. The mode is given by `stats.mode([0, 1, 2, 3, 3, 3, 5])[0]=`{glue:}`mode` (access the counts using `stats.mode(...)[1]`).

Less famous quantiles than the median are given by, for example for $q=0.25$,

In [ ]:
np.quantile(data, 0.25)

As with **pandas**, **numpy** and **scipy** work on scalars, vectors, matrices, and tensors: you just need to specify the axis that you'd like to apply a function to:

In [ ]:
data = np.fromfunction(lambda i, j: i + j, (3, 6), dtype=int)
data

In [ ]:
np.mean(data, axis=0)

Remember that, for discrete data points, the $k$th (unnormalised) moment is

$$
m_k = \frac{1}{n}\displaystyle\sum_{i=1}^{n} \left(x_i - \bar{x}\right)^k
$$

To compute this use scipy's `stats.moment(a, moment=1)`. For instance for the kurtosis ($k=4$), it's

In [ ]:
stats.moment(data, moment=4, axis=1)

Covariances are found using `np.cov`.

In [ ]:
np.cov(np.array([[0, 1, 2], [2, 1, 0]]))

Note that, as expected, the $C_{01}$ term is -1 as the vectors are anti-correlated.

## Randomness

In this section, we'll see how to generate random numbers. Numpy’s random number routines produce pseudo-random numbers, which is the best that any computer can do. If you care about how, the default PRNG (psuedo-random number generator) is a 64-bit Permuted Congruential Generator, though you can access other generators too.

The way to use the numpy PRNG is to call it in like this:

In [ ]:
from numpy.random import default_rng
rng = default_rng()
x_vals = rng.random(size=2)
x_vals

In the above, `rng` is an object that you can call many random number generating functions on. Here we just asked for 2 values drawn from between 0 and 1.

Another really useful random generator provides integers and is called `integers`. Let's see this but in the case where we're asking for a more elaborately shaped output array, a 3x3x2 dimensional tensor:

In [ ]:
min_int = 1
max_int = 20
rng.integers(min_int, max_int, size=(3, 3, 2))

One random function that is incredibly useful is `choice`, which returns a random selection from another type of object. Here, we show this by passing a list of letters and asking for two of them to be picked randomly:

In [ ]:
rng.choice(['a', 'b', 'c', 'd', 'e', 'f'], size=2)

This choice can also be made with a given probability. Let's make a very large number of draws with an exponentially falling probability and see what we get!

In [ ]:
num_draws = 1000
# Create 6 values spread across several orders of magnitude
prob = np.logspace(0, -3, num=6)
# Normalise this to 1
prob = prob/sum(prob)
# Choose the letters
letter_choices = rng.choice(['a', 'b', 'c', 'd', 'e', 'f'], size=num_draws, p=prob)

To make it easy to see what happened, we'll use the in-built collections library's `Counter` function to go from a long list of all of the letters to a dictionary of letters and counts of how frequently they occurred:

In [ ]:
from collections import Counter
counts = Counter(letter_choices)
plt.bar(counts.keys(), counts.values());

As expected, 'a' was chosen many more times than 'b', and so on. In fact, if we divided the counts by `num_draws`, we would find that the probability of each letter was converging toward the probabilities we provided in `prob`.

Another useful random function to know about is `shuffle`, and you can probably guess what it does! But note that it does the shuffling to the list you put in, rather than returning a new, modified list. Here's an example:

In [ ]:
plain_list = ['This', 'list', 'is', 'well', 'ordered.']
rng.shuffle(plain_list)
plain_list

If you are using **pandas** for your analysis, then it comes with random sampling methods built in under the guise of `df.sample()` for a dataframe `df`. This has keywords for number of samples (`n`) **or** fraction of all rows to sample (`frac`) and whether to use `weights=`.

### Reproducibility

If you need to create random numbers reproducibly, then you can do it by setting a seed value like this:

In [ ]:
from numpy.random import Generator, PCG64
seed_for_prng = 54321
prng = Generator(PCG64(seed_for_prng))
prng.integers(0, 10, size=2)

In [ ]:
prng = Generator(PCG64(seed_for_prng))
prng.integers(0, 10, size=2)

The seed tells the generator where to start (PCG64 is the default generator), so by passing the same seed in we can make the random numbers begin in the same place.

The `prng` above can also be passed to the **pandas** sample function as a keyword argument.

### Random numbers drawn from distributions

Using **numpy**, we can often draw samples from distributions using the `prng.distribution` syntax. One of the most common distributions you might like to draw from is the uniform, for example

$$
x \thicksim \mathcal{U}(0, 10)
$$

with, here, a minimum of 0 and a maximum of 10. Here's the code:



In [ ]:
prng.uniform(low=0, high=10, size=3)

Let's see how to draw from one other important distribution function: the Gaussian, or normal, distribution $x \thicksim \mathcal{N}\left(\mu, \sigma\right)$ and check that it looks right. We'll actually do two different ones: a standard normal, with $\mu=0$ and $\sigma=1$, and a shifted, relaxed one with different parameters.

In [ ]:
def gauss(x):
    """Analytical Gaussian."""
    return (1/np.sqrt(2*np.pi))*np.exp(-0.5*x**2)


# Make the random draws
num_draws = 10000
vals = prng.standard_normal(num_draws)

# Get analytical solution
x_axis_vals = np.linspace(-3, 3, 300)
analytic_y = gauss(x_axis_vals)

# Random draws of shifted/flatter dist
mu = 0.5
sigma = 2
vals_shift = prng.normal(loc=mu, scale=sigma, size=num_draws)

fig, ax = plt.subplots()
ax.plot(x_axis_vals, analytic_y, label='Std norm: analytical', lw=3)
ax.hist(vals, bins=50, label='Std norm: generated', density=True, alpha=0.8)
ax.hist(vals_shift, bins=50, label=f'Norm: $\mu$={mu}, $\sigma$={sigma}', density=True, alpha=0.8)
ax.legend(frameon=False)
plt.show()

What if there isn't a pre-built distribution available? Well, first, that would be surprising, because there's a huge number of them! (More on that in a moment.) But a quick way to get around this is to plug random numbers into the inverse cumulative distribution function (assuming it exists) of your distribution function, ie you plug random numbers $r$ into $\text{cdf}^{-1}(r)$ in order to generate $x \thicksim \text{pdf}$.

## Probability

We've already seen some bits of probability and how to do them already; options for probabilistic sampling popped up in `df.sample` and `choices`, and, of course, probability density functions like the Gaussian have an intimate relationship with probability. Let's take a step back now and discuss what we mean by probability and random variables.

A random variable is a function that maps a sample space $S$ into the real numbers $\mathbb{R}$, i.e. $X:S\rightarrow\mathbb{R}$. Discrete random variables have a finite (or countably infinite) number of values (think integers); continuous random variables have infinitely many values (think a Gaussian).

### Probability mass functions

Even armed with this, we can do some very simple simulations of probability. We start with the canonical example, rolling a 6-sided die...

..but we want to make it a *bit* more exciting than that! Let's see two die, one that's fair (equal probability of getting any value in 1 to 6) and one that's loaded (in this case, we'll make a 6 twice as likely as other values).

A brief recap on probability: in this case our sample space is $S = {1, 2, 3, 4, 5, 6}$. To work out the probability of a particular value, it's going to be

$$
\frac{\text{Counts}_s}{\text{Total counts}}
$$

with $s \in {1, 2, 3, 4, 5, 6}$.

To simualte this, we'll use the `choice` function fed with the six values, 1 to 6, on standard dice. Then we'll count the occurrences of each, creating a dictionary of keys and values with `Counter`, and then plot those.

In [ ]:
import collections

throws = 1000
die_vals = np.arange(1, 7)
probabilities = [1/7, 1/7, 1/7, 1/7, 1/7, 2/7]
fair_throws = prng.choice(die_vals, size=throws)
load_throws = prng.choice(die_vals, size=throws, p=probabilities)


def throw_list_to_array(throw_list):
    # Count frequencies of what's in throw list but order the dictionary keys
    counts_dict = collections.OrderedDict(sorted(Counter(throw_list).items()))
    # Turn the key value pairs into a numpy array
    array = np.array([list(counts_dict.keys()), list(counts_dict.values())], dtype=float)
    # Divide counts per value by num throws
    array[1] = array[1]/len(throw_list)
    return array


counts_fair = throw_list_to_array(fair_throws)
counts_load = throw_list_to_array(load_throws)

fig, ax = plt.subplots()
ax.scatter(counts_fair[0],
           counts_fair[1],
           color='b',
           label='Fair')
ax.scatter(counts_load[0],
           counts_load[1],
           color='r',
           label='Loaded')
ax.set_xlabel('Die value')
ax.set_ylabel('Probability')
ax.axhline(1/6, color='k', alpha=0.3, linestyle='-.', lw=0.5)
ax.legend(frameon=True, loc='upper left')
ax.set_ylim(0., 0.4);

To work out the probability based on the simulation, we've divided the number of throws per value by the total number of throws. You can see that with so many throws, there's quite a wedge between the chance of obtaining a six in both cases. Meanwhile, the fair die is converging to the dotted line, which is $1/6$. Note that because of the individual probabilities summing to unity, a higher probability of a six on the loaded die means that values 1 to 5 must have a lower probability than with the fair die; and you can see that emerging in the chart too.

In doing this for every possible outcome, we're effectively estimating a probability mass function (if we rescale by the number of throws); an object that tells us the probabilty mass given to specific outcomes. The more precise defintion of a probability mass function (or pmf) is

$$
p(x_i) = P(X=x_i) = P(\underbrace{\{s\in S\ |\ X(s) = x_i\}}_{\text{set of outcomes resulting in}\ X=x_i}).
$$

It has a few key properties. The probability of all outcomes sum to 1, ie $\displaystyle{\sum_{x_i} p(x_i)}=1$, the probabilities satisfy $p(x_i) \geq 0  \quad\forall x_i$, and $P(X \in A) = \displaystyle\sum_{x_i \in A} p(x_i)$.

Another useful object is the cumulative distribution function, which is defined generally as $\text{cdf}(x) = P(X \leq x)\quad \forall x \in \mathbb{R}$. For probability mass functions, this becomes

$$
\text{cdf}(x) = P(X\leq x) = \sum_{x_i\leq x} p(x_i)
$$

Let's estimate the probability mass functions for our dice using the `cumsum` function: 

In [ ]:
fig, ax = plt.subplots()
ax.plot(counts_fair[0],
           np.cumsum(counts_fair[1]),
           color='b',
           label='Fair')
ax.plot(counts_load[0],
        np.cumsum(counts_load[1]),
        color='r',
           label='Loaded')
ax.set_xlabel('Die value')
ax.set_ylabel('Cumulative distribution function')
ax.axhline(1, color='k', alpha=0.3, linestyle='-.', lw=0.5)
ax.legend(frameon=True, loc='lower right')
ax.set_ylim(0., 1.2);

We can see that the cumulative distribution function also tells a story about what's going on; namely, there is a lower gradient up to $i=6$, followed by a higher gradient. The two distributions are visually distinct.

Bernoulli and other famous discrete distributions.

### Probability density functions

PDFs are to continuous variables what PMFs are to discrete variables, though there are some important differences that trip up even the most careful.
This is different to a probability density function!!!